<a href="https://colab.research.google.com/github/tlchampion/drybeans/blob/main/notebooks/db_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feature-engine

!pip uninstall xgboost -y
!pip install -U xgboost

# !pip uninstall scikit-learn -y
# !pip install -U scikit-learn

!pip install dython
!pip install scikit-optimize

Found existing installation: xgboost 1.6.0
Uninstalling xgboost-1.6.0:
  Successfully uninstalled xgboost-1.6.0
  Using cached xgboost-1.6.0-py3-none-manylinux2014_x86_64.whl (193.7 MB)


In [12]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn

from scipy.stats import chi2_contingency

from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import DropConstantFeatures
from feature_engine.selection import SmartCorrelatedSelection
from dython import nominal
from feature_engine.encoding import OrdinalEncoder as oe, MeanEncoder, CountFrequencyEncoder, RareLabelEncoder, OneHotEncoder as ohe
from feature_engine.imputation import ArbitraryNumberImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.creation import CombineWithReferenceFeature
from feature_engine.outliers import ArbitraryOutlierCapper
import seaborn as sns
import math

import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler,FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import os
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from joblib import dump, load
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, explained_variance_score, r2_score, mean_squared_error
from skopt import BayesSearchCV
import xgboost as xgb
from skopt.space import Real, Categorical, Integer, Space
from sklearn.preprocessing import FunctionTransformer



In [3]:
basedir = '/content/drive/MyDrive/Projects/drybeans'
try:
    os.makedirs(os.path.join(basedir, "data/02_interim"))
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(os.path.join(basedir, "data/03_processed"))
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(os.path.join(basedir, "models"))
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(os.path.join(basedir, "models/model1"))
except FileExistsError:
    # directory already exists
    pass




In [11]:
datapath = os.path.join(basedir, "data/01_raw/Dry_Bean_Dataset.xlsx") 
data = pd.read_excel(datapath, engine='openpyxl')


In [10]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653248,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON
